# Movies Recomendation 

In [1]:
# Importando bibliotecas que serão utilizadas

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
# Importando dataframes que serão utilizados

df_movies = pd.read_csv(r"G:\Meu Drive\My Repositories\Datasets_for_data_science\df's-db's-ds's\databases\datasets\ml-latest-small\\movies.csv")
df_ratings = pd.read_csv(r"G:\Meu Drive\My Repositories\Datasets_for_data_science\df's-db's-ds's\databases\datasets\ml-latest-small\\ratings.csv")
#df_tags = pd.read_csv(r"G:\Meu Drive\My Repositories\Datasets_for_data_science\df's-db's-ds's\databases\datasets\ml-latest-small\\tags.csv")

In [3]:
# Alterando o dataframe para melhor uso 

df_movies = df_movies.set_index(df_movies['movieId'])
df_movies = df_movies.drop(columns = ['movieId'])
df_movies['mean_ratings'] = df_ratings.groupby('movieId').mean()['rating']
df_movies['total_rates'] = df_ratings['movieId'].value_counts()

df_ratings = df_ratings.set_index('userId')

In [4]:
# Função para calcular a distância entre dois pontos

def vetor_dist(a,b):
  return np.linalg.norm(a - b)

In [5]:
# Função para capturar as notas de um determinado usuário

def user_grade(user_id):
    user_grades = df_ratings.query(f'userId == {user_id}')
    user_grades = user_grades[['movieId', 'rating']].set_index('movieId')

    return user_grades

In [5]:
# Função para calcular a distância entre dois usuários

def users_dist(user_id_01, user_id_02, min = 5):
  user_grade_01 = user_grade(user_id_01)
  user_grade_02 = user_grade(user_id_02)
  grade = user_grade_01.join(user_grade_02, lsuffix = f'_{user_id_01}', rsuffix = f'_{user_id_02}').dropna()
  dist = vetor_dist(grade[f'rating_{user_id_01}'], grade[f'rating_{user_id_02}'])
  
  if (len(grade) < min):
    return [user_id_01, user_id_02, None]
    
  return [user_id_01, user_id_02, dist]

KeyboardInterrupt: 

In [ ]:
# Função para calcular a distância de um usuário com todos os outros

def all_users_dist(user_id):
  users = df_ratings.index.unique()
  dists = [users_dist(user_id, user) for user in users]
  df_users_dist = pd.DataFrame(dists, columns = ['Principal Id', 'Index', 'Distance']).set_index('Index')
  df_users_dist = df_users_dist.drop(1)
  
  return df_users_dist

In [ ]:
all_users_dist(1).head()

,Principal Id,Distance
Index,,
2,1,NaN
3,1,8.200610
4,1,11.135529
5,1,3.741657
6,1,8.602325


In [ ]:
# Função para indicar os usuários mais proximos de um usuário determinado 

def most_near(user_id):
  df_users_dist = all_users_dist(user_id)
  near = df_users_dist.sort_values('Distance', ascending = True).dropna()

  return near

In [ ]:
most_near(1).head()

,Principal Id,Distance
Index,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
9,1,1.000000
49,1,1.000000


In [ ]:
# Função para recomendar filmes baseado nos usuários mais proximos 

def recommend_movies(user_id, k = 100, m = 10):

    near_users = most_near(user_id).head(k).index.values

    near_user_data = df_ratings.loc[user_id].sort_values('rating', ascending = False).head(m).drop(columns= ['timestamp']).set_index('movieId')
    near_user_data = {'user': user_id, 'movies': [i for i in near_user_data.index.values], 'rates': [r for r in near_user_data.rating.values]}
    data = pd.DataFrame().from_dict(near_user_data).set_index('user')

    for c in near_users:
        if c == near_users[0]:
            pass

        near_user_data = df_ratings.loc[c].sort_values('rating', ascending = False).head(m).drop(columns= ['timestamp']).set_index('movieId')
            
        near_user_data = {'user': c, 'movies': [i for i in near_user_data.index.values], 'rates': [r for r in near_user_data.rating.values]}

        near_user_data = pd.DataFrame().from_dict(near_user_data).set_index('user')
            
        data = pd.concat([data, near_user_data])

    most_recommend_df = data.movies.value_counts().to_frame('rates_counts').sort_values('rates_counts', ascending = False)

    for c in user_grade(user_id).index.values:
        if c in most_recommend_df.index.values:
            most_recommend_df = most_recommend_df.drop(c)

    most_recommend_indexs = [k for k in most_recommend_df.index.values]
    most_recommend_df['mean_rates'] = [data.loc[data['movies'] == c].rates.mean() for c in most_recommend_indexs]

    most_recommend_movies = [df_movies.loc[i].title for i in most_recommend_indexs]
    most_recommend_df['movies'] = most_recommend_movies

    most_recommend_df = most_recommend_df.sort_values(['rates_counts', 'mean_rates'], ascending = False).head(m)

    recommend_movies = [m for m in most_recommend_df.movies]

    return recommend_movies

In [ ]:
# Simulando quais as recomendações para um usuário novo

data = [102084, 5],[153, 4],[3793, 4],[5349, 5],[33794, 5],[60069, 4],[2706, 4],[723998, 4.5],[59315, 5]

def new_user(data):

    index = df_ratings.index.max()+1
    new_user = pd.DataFrame(data, columns=['movieId', 'rating'])
    new_user['userId'] = index
    new_user = new_user.set_index('userId')

    new_df_ratings = pd.concat([df_ratings, new_user])

    return new_df_ratings

In [ ]:
df_ratings.tail()

,movieId,rating,timestamp
userId,,,
610,166534,4.0,1493848402
610,168248,5.0,1493850091
610,168250,5.0,1494273047
610,168252,5.0,1493846352
610,170875,3.0,1493846415


In [ ]:
df_ratings = new_user(data)
df_ratings.tail()

,movieId,rating,timestamp
userId,,,
611,33794,5.0,NaN
611,60069,4.0,NaN
611,2706,4.0,NaN
611,723998,4.5,NaN
611,59315,5.0,NaN


In [ ]:
recommend_movies(611)

['Toy Story (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Dark Knight, The (2008)',
 'Shawshank Redemption, The (1994)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'American History X (1998)',
 'Fight Club (1999)',
 'Back to the Future (1985)',
 'Monty Python and the Holy Grail (1975)']

# MultiLabel

In [29]:
# Importando bibliotecas que serão utilizadas 

import pandas as pd
import numpy as np 

from string import punctuation

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, classification_report, hamming_loss

from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.adapt import MLkNN
# Cursores

SEED = 13
np.random.seed(SEED)

# Modelos  Sklearn

LogisticRegression = LogisticRegression(solver = 'lbfgs')
OneVsRestClassifier = OneVsRestClassifier(LogisticRegression)
vetorizer = TfidfVectorizer(max_features = 5000, max_df = 0.85)

# Modelo Scikit-Multilearn

ClassifierChain = ClassifierChain(LogisticRegression)
BinaryRelevance =  BinaryRelevance(LogisticRegression)
MLkNN = MLkNN()

In [2]:
# Importando dataframes que serão utilizados 
 
df_questions = pd.read_csv(r"G:\Meu Drive\My Repositories\Datasets_for_data_science\df's-db's-ds's\databases\datasets\\stackoverflow_perguntas.csv")

In [3]:
# Alterando dataframe para melhor utilização 

rename_columns = {
    'Perguntas': 'questions',
    'Tags': 'tags'
}

df_questions = df_questions.rename(columns = rename_columns)

In [4]:
# Criando um vetor com todos os labels possiveis para o categorizador

def catch_unique(label):
    labels_unique = []

    for tags in label.unique():
        for tag in tags.split():
            if tag not in labels_unique:
                labels_unique.append(tag.lower())

    return labels_unique
    
labels_unique = catch_unique(df_questions.tags)

In [5]:
# Criando uma coluna para cada label possível no dataframe

def column_label(df, label):
    labels_unique = catch_unique(label)

    for tag in labels_unique:
        temp = []
        for line in label:
            if tag in line: temp.append(1)
            else: temp.append(0)
        df[tag] = temp

column_label(df_questions, df_questions.tags)

In [6]:
lista_zip_tags = list(zip(df_questions[labels_unique[0]], df_questions[labels_unique[1]], df_questions[labels_unique[2]], df_questions[labels_unique[3]]))

df_questions["labels"] = lista_zip_tags

In [7]:
# Separando dados de treino e teste para o nosso modelo de classificação 

train_x, test_x, train_y, test_y = train_test_split(df_questions.questions, df_questions.labels, test_size = 0.3, stratify = df_questions.labels)

#y_true

y_true = test_y

# Vetorizando o X para que o modelo possa treinar matemáticamente

vetorizer.fit(df_questions.questions)

train_x = vetorizer.transform(train_x)
test_x = vetorizer.transform(test_x)

# Transformando em array numpy para que o modelo possa compreender as tags

train_y = np.asarray(list(train_y))
test_y = np.asarray(list(test_y))

In [8]:
## OneVsRestClassiier

# Treinando o modelo 

OneVsRestClassifier.fit(train_x, train_y)

# Testando o modelo 

predict = OneVsRestClassifier.predict(test_x)

# Medindo o modelo 

accuracy = accuracy_score(test_y, predict)
report = classification_report(test_y, predict)
hamming_loss = hamming_loss(test_y, predict)

C:\Users\guilh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
print(accuracy)
print(report)
(1 - hamming_loss) * 100 

0.39802834257547753
              precision    recall  f1-score   support

           0       0.97      0.31      0.47       192
           1       0.71      0.61      0.66       733
           2       0.69      0.61      0.65       704
           3       1.00      0.35      0.52       279

   micro avg       0.73      0.54      0.62      1908
   macro avg       0.84      0.47      0.58      1908
weighted avg       0.77      0.54      0.62      1908
 samples avg       0.55      0.55      0.53      1908



80.8071472581639

In [10]:
## ClassifierChain

# Treinando o modelo 

ClassifierChain.fit(train_x, train_y)

# Testando o modelo 

predict = ClassifierChain.predict(test_x)

# Medindo o modelo 

accuracy = accuracy_score(test_y, predict)
report = classification_report(test_y, predict)
#hamming_loss = hamming_loss(tes_y, predict)

In [20]:
print(accuracy)
print(report)
#(1 - hamming_loss) * 100 

0.4929143561306223
              precision    recall  f1-score   support

           0       0.97      0.31      0.47       192
           1       0.68      0.71      0.69       733
           2       0.57      0.55      0.56       704
           3       0.80      0.48      0.60       279

   micro avg       0.66      0.58      0.62      1908
   macro avg       0.75      0.51      0.58      1908
weighted avg       0.69      0.58      0.61      1908
 samples avg       0.66      0.59      0.61      1908



In [26]:
## Binary Relevence

# Treinando o modelo 

BinaryRelevance.fit(train_x, train_y)

# Testando o modelo 

predict = BinaryRelevance.predict(test_x)

# Medindo o modelo 

accuracy = accuracy_score(test_y, predict)
report = classification_report(test_y, predict)
hamming_loss = hamming_loss(test_y, predict)

C:\Users\guilh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
print(accuracy)
print(report)
(1 - hamming_loss) * 100 

0.39802834257547753
              precision    recall  f1-score   support

           0       0.97      0.31      0.47       192
           1       0.71      0.61      0.66       733
           2       0.69      0.61      0.65       704
           3       1.00      0.35      0.52       279

   micro avg       0.73      0.54      0.62      1908
   macro avg       0.84      0.47      0.58      1908
weighted avg       0.77      0.54      0.62      1908
 samples avg       0.55      0.55      0.53      1908



80.8071472581639

In [30]:
## Binary Relevence

# Treinando o modelo 

MLkNN.fit(train_x, train_y)

# Testando o modelo 

predict = MLkNN.predict(test_x)

# Medindo o modelo 

accuracy = accuracy_score(test_y, predict)
report = classification_report(test_y, predict)
hamming_loss = hamming_loss(test_y, predict)

C:\Users\guilh\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\guilh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
print(accuracy)
print(report)
(1 - hamming_loss) * 100 

0.3672211953173136
              precision    recall  f1-score   support

           0       0.77      0.47      0.59       192
           1       0.62      0.52      0.57       733
           2       0.59      0.70      0.64       704
           3       0.60      0.41      0.49       279

   micro avg       0.61      0.57      0.59      1908
   macro avg       0.64      0.53      0.57      1908
weighted avg       0.62      0.57      0.58      1908
 samples avg       0.55      0.57      0.54      1908



76.66358595194085

# Fraud detection

In [33]:
# Importando bibliotecas que serão utilizadas 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import itertools 

from datetime import timedelta

from IPython.display import display

from scipy import special 

In [34]:
# Importando dataframes que serão utilizados 

df_fraud = pd.read_csv(r"G:\Meu Drive\My Repositories\Datasets_for_data_science\df's-db's-ds's\databases\datasets\fraud-detection\\Fraud_Data.csv", header = 0)
df_isp = pd.read_csv(r"G:\Meu Drive\My Repositories\Datasets_for_data_science\df's-db's-ds's\databases\datasets\fraud-detection\\IpAddress_to_Country.csv", header = 0)

In [35]:
# Curso com falhas

# Marketing Digital

In [175]:
# Importando bibliotecas que serão utilizadas

import pandas as pd 
import numpy as np
import json

In [178]:
# Importando bibliotecas que serão utilizadas

df_marketing = pd.read_csv(r"G:\Meu Drive\My Repositories\Datasets_for_data_science\df's-db's-ds's\databases\datasets\\marketing_digital.csv", header = 0, 
                           dtype = {'date': object, 'fullVisitorId': object, 'visitId': object})

# Transformando colunas com valores = dicts em colunas novas

columns_dicts = ['device', 'geoNetwork', 'totals', 'trafficSource']

for coluna in columns_dicts:
    df_marketing = df_marketing.join(pd.DataFrame([json.loads(linha) for linha in df_marketing[coluna]]))
    df_marketing.drop(columns = coluna, axis = 1, inplace = True)


# Dropando colunas que retornam um erro 

df_marketing.drop(columns = 'adwordsClickInfo', axis = 1, inplace = True)

In [179]:
val = 0
for coluna in df_marketing.columns:
    unique = len(df_marketing[coluna].unique())
    print(f'{coluna} possui: {unique} itens distintos')
    
    if unique == 1:
        df_marketing.drop(columns = coluna, axis = 1, inplace = True)
    
    val += 1

channelGrouping possui: 8 itens distintos
date possui: 366 itens distintos
fullVisitorId possui: 9997 itens distintos
sessionId possui: 12276 itens distintos
socialEngagementType possui: 1 itens distintos
visitId possui: 12272 itens distintos
visitNumber possui: 50 itens distintos
visitStartTime possui: 12279 itens distintos
browser possui: 22 itens distintos
browserVersion possui: 1 itens distintos
browserSize possui: 1 itens distintos
operatingSystem possui: 12 itens distintos
operatingSystemVersion possui: 1 itens distintos
isMobile possui: 2 itens distintos
mobileDeviceBranding possui: 1 itens distintos
mobileDeviceModel possui: 1 itens distintos
mobileInputSelector possui: 1 itens distintos
mobileDeviceInfo possui: 1 itens distintos
mobileDeviceMarketingName possui: 1 itens distintos
flashVersion possui: 1 itens distintos
language possui: 1 itens distintos
screenColors possui: 1 itens distintos
screenResolution possui: 1 itens distintos
deviceCategory possui: 3 itens distintos
con

In [187]:
numeric_columns = ['hits', 'pageviews', 'bounces', 'newVisits', 'transactionRevenue']
for coluna in numeric_columns:
    df_marketing[coluna] = pd.to_numeric(df_marketing[coluna])

In [192]:
df_marketing.transactionRevenue = df_marketing.transactionRevenue/1000000
df_marketing.transactionRevenue.fillna(0, inplace=True)

In [213]:
important_columns = ['fullVisitorId', 'hits', 'pageviews', 'bounces', 'transactionRevenue']
df_visitors = df_marketing.groupby('fullVisitorId', as_index = False)[important_columns].sum()

In [205]:
df_marketing.head(3)

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,browser,operatingSystem,isMobile,...,newVisits,transactionRevenue,campaign,source,medium,keyword,isTrueDirect,referralPath,adContent,campaignCode
0,Organic Search,20160902,9674781571160116268,9674781571160116268_1472804607,1472804607,1,1472804607,Chrome,Windows,False,...,1.0,0.0,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN
1,Organic Search,20160902,8590648239310839049,8590648239310839049_1472835928,1472835928,1,1472835928,Chrome,Macintosh,False,...,1.0,0.0,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN
2,Affiliates,20160902,9772828344252850532,9772828344252850532_1472856802,1472856802,1,1472856802,Chrome,Macintosh,False,...,1.0,0.0,Data Share Promo,Partners,affiliate,NaN,NaN,NaN,NaN,NaN


In [214]:
df_visitors.head(3)

,fullVisitorId,hits,pageviews,bounces,transactionRevenue
0,0002365800130207040,1,1,1.0,0.0
1,0010286039787739137,4,3,0.0,0.0
2,0011056874471185769,2,2,0.0,0.0
